# LangGraph Tutorial: Conversational Context

## Objective
Understand how MessagesState maintains conversation history across multiple turns, enabling the agent to reference previous exchanges.

## What You'll Learn
1. How state preserves conversation history
2. Multi-turn conversation patterns
3. Context-aware tool usage (referencing previous results)
4. Starting fresh conversations vs continuing
5. Message accumulation and context growth

## Prerequisites
- Completed: Notebook 08 (Sequential Execution)
- Understanding of message flow in tool-calling agents

---

## Section 1: The Conversational Context Pattern

### How Context Works in LangGraph

```
Turn 1: User asks about currency conversion
  → State: [Human1, AI1(tool), Tool1, AI1(response)]

Turn 2: User references "that amount" for EMI
  → State: [Human1, AI1, Tool1, AI1, Human2, AI2(tool), Tool2, AI2(response)]
  → LLM sees ENTIRE history, understands "that amount"

Turn 3: User asks "what did you calculate?"
  → State: [...all previous..., Human3, AI3(response)]
  → LLM can recall any past exchange
```

### Reference Point: State Persistence Pattern

```python
# The key pattern for multi-turn conversations:
state = {"messages": [HumanMessage("Turn 1")]}
state = app.invoke(state)  # State grows with responses

state["messages"].append(HumanMessage("Turn 2"))  # Add next query
state = app.invoke(state)  # LLM sees FULL history
```

### Reference Point: Message Accumulation

| Turn | User Action | State Size | LLM Context |
|------|-------------|------------|-------------|
| 1 | Initial query | 1 → 4 messages | Current query only |
| 2 | Follow-up | 5 → 8+ messages | All of Turn 1 + Turn 2 |
| 3 | Reference past | 9+ messages | Complete history |

**Key Insight:** State only grows, never shrinks. The LLM sees everything!

---

## Section 2: Setup

Build the financial assistant graph.

In [ ]:
# Core imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv
from typing import Literal

load_dotenv("../../.env")
print("✅ Environment loaded")

In [ ]:
# Define tools
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Use this tool when users need to convert monetary amounts between
    different currencies. Supports USD, EUR, GBP, INR, and JPY.
    """
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates or to_currency not in exchange_rates:
        return f"Error: Unsupported currency"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    """
    if principal <= 0 or annual_interest_rate < 0 or tenure_months <= 0:
        return "Error: Invalid input parameters"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")

In [ ]:
# Initialize LLM and build graph
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

tools = [currency_converter, emi_calculator]
llm_with_tools = llm.bind_tools(tools)

def call_llm(state: MessagesState):
    """Agent node: Calls LLM with current messages."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router: Check if agent wants to use tools."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

# Build graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "agent")

app = workflow.compile()
print("✅ Graph compiled")

---

## Section 3: Multi-Turn Conversation Example

We'll simulate a realistic conversation where each turn builds on previous context.

**Conversation Plan:**
1. Turn 1: Ask about currency conversion
2. Turn 2: Reference "that amount" for EMI calculation
3. Turn 3: Ask about previous calculations (no tool needed)

In [ ]:
# Initialize conversation state
conversation = {
    "messages": []
}

print("Starting Multi-Turn Conversation")
print("=" * 80)

### Turn 1: Initial Query (Currency Conversion)

In [ ]:
# Turn 1: Currency conversion
print("\n" + "─" * 80)
print("TURN 1")
print("─" * 80)

user_query_1 = "Convert 50000 USD to INR"
print(f"👤 User: {user_query_1}")

# Add user message and invoke
conversation["messages"].append(HumanMessage(content=user_query_1))
conversation = app.invoke(conversation)

print(f"\n🤖 Assistant: {conversation['messages'][-1].content}")
print(f"\n📊 State size: {len(conversation['messages'])} messages")

### Turn 2: Reference Previous Result

In [ ]:
# Turn 2: EMI calculation referencing previous conversion
print("\n" + "─" * 80)
print("TURN 2")
print("─" * 80)

user_query_2 = "Calculate EMI for that INR amount at 8.5% for 60 months"
print(f"👤 User: {user_query_2}")
print("\n   Note: 'that INR amount' references Turn 1's result!")

# Add user message and invoke
conversation["messages"].append(HumanMessage(content=user_query_2))
conversation = app.invoke(conversation)

print(f"\n🤖 Assistant: {conversation['messages'][-1].content}")
print(f"\n📊 State size: {len(conversation['messages'])} messages")

### Turn 3: Ask About Previous Calculations (No Tool Needed)

In [ ]:
# Turn 3: Simple recall question
print("\n" + "─" * 80)
print("TURN 3")
print("─" * 80)

user_query_3 = "What was the monthly EMI you calculated?"
print(f"👤 User: {user_query_3}")
print("\n   Note: Agent should answer from memory, no tool needed!")

# Add user message and invoke
conversation["messages"].append(HumanMessage(content=user_query_3))
conversation = app.invoke(conversation)

print(f"\n🤖 Assistant: {conversation['messages'][-1].content}")
print(f"\n📊 State size: {len(conversation['messages'])} messages")

---

## Section 4: Examine the Conversation State

In [ ]:
print("COMPLETE CONVERSATION STATE")
print("=" * 80)

turn_num = 0
for i, msg in enumerate(conversation["messages"], 1):
    if isinstance(msg, HumanMessage):
        turn_num += 1
        print(f"\n{'─' * 80}")
        print(f"TURN {turn_num}")
        print(f"{'─' * 80}")
        print(f"[{i}] 👤 USER: {msg.content[:60]}..." if len(msg.content) > 60 else f"[{i}] 👤 USER: {msg.content}")
        
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"[{i}] 🤖 AGENT: Calling {msg.tool_calls[0]['name']}")
        else:
            preview = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
            print(f"[{i}] 🤖 AGENT: {preview}")
            
    elif isinstance(msg, ToolMessage):
        first_line = msg.content.split('\n')[0]
        print(f"[{i}] 🔧 TOOL: {first_line}")

print(f"\n{'=' * 80}")
print(f"Total messages in state: {len(conversation['messages'])}")

### Reference Point: How Context Enables Understanding

| Turn | User Said | LLM Understood | Because |
|------|-----------|----------------|--------|
| 2 | "that INR amount" | 4,156,000 INR | Saw Turn 1's ToolMessage |
| 3 | "the EMI you calculated" | Previous EMI value | Saw Turn 2's ToolMessage |

**The LLM doesn't "remember"—it sees the ENTIRE history every time!**

---

## Section 5: Second Example - Car Purchase Planning

A more realistic multi-turn scenario.

In [ ]:
# Start fresh conversation
car_conversation = {"messages": []}

print("SCENARIO: Car Purchase Planning")
print("=" * 80)

In [ ]:
# Turn 1: Budget conversion
print("\nTurn 1")
print("-" * 40)
query1 = "I have a budget of 2000000 INR for a car. What's that in USD?"
print(f"👤 User: {query1}")

car_conversation["messages"].append(HumanMessage(content=query1))
car_conversation = app.invoke(car_conversation)

print(f"🤖 Assistant: {car_conversation['messages'][-1].content}")

In [ ]:
# Turn 2: EMI for full budget
print("\nTurn 2")
print("-" * 40)
query2 = "If I take a loan for my full budget at 9% for 5 years, what's my monthly payment?"
print(f"👤 User: {query2}")
print("   (References 'my full budget' = 2,000,000 INR from Turn 1)")

car_conversation["messages"].append(HumanMessage(content=query2))
car_conversation = app.invoke(car_conversation)

print(f"\n🤖 Assistant: {car_conversation['messages'][-1].content}")

In [ ]:
# Turn 3: What-if scenario
print("\nTurn 3")
print("-" * 40)
query3 = "What if I reduce the tenure to 3 years instead?"
print(f"👤 User: {query3}")
print("   (References same loan amount and rate, just different tenure)")

car_conversation["messages"].append(HumanMessage(content=query3))
car_conversation = app.invoke(car_conversation)

print(f"\n🤖 Assistant: {car_conversation['messages'][-1].content}")

In [ ]:
# Turn 4: Comparison question
print("\nTurn 4")
print("-" * 40)
query4 = "How much more would I pay monthly with the 3-year plan compared to 5 years?"
print(f"👤 User: {query4}")
print("   (Requires comparing results from Turn 2 and Turn 3)")

car_conversation["messages"].append(HumanMessage(content=query4))
car_conversation = app.invoke(car_conversation)

print(f"\n🤖 Assistant: {car_conversation['messages'][-1].content}")
print(f"\n📊 Final state size: {len(car_conversation['messages'])} messages")

---

## Section 6: Starting a Fresh Conversation

To reset context, simply create a new state object.

In [ ]:
# New conversation (no context from previous)
fresh_conversation = {
    "messages": [
        HumanMessage(content="Convert 500 EUR to GBP")
    ]
}

result_fresh = app.invoke(fresh_conversation)

print("FRESH CONVERSATION (New State)")
print("=" * 80)
print(f"Messages in state: {len(result_fresh['messages'])}")
print(f"\n🤖 Response: {result_fresh['messages'][-1].content}")
print("\n✅ No context from previous car purchase conversation!")
print("   (It doesn't know about 2,000,000 INR or any EMI calculations)")

### Reference Point: Fresh vs Continued Conversations

```python
# CONTINUE existing conversation:
state["messages"].append(HumanMessage("Next question"))
state = app.invoke(state)  # Sees ALL previous messages

# START fresh conversation:
new_state = {"messages": [HumanMessage("First question")]}
new_state = app.invoke(new_state)  # No previous context
```

---

## Section 7: Context-Aware Behavior Analysis

### Reference Point: What the LLM Can Do With Context

| Capability | Example | How It Works |
|------------|---------|-------------|
| **Reference values** | "that amount" | Finds value in previous ToolMessage |
| **Recall calculations** | "what was the EMI?" | Reads from conversation history |
| **Modify parameters** | "change tenure to 3 years" | Keeps other params, updates one |
| **Compare results** | "difference between plans" | Analyzes multiple past tool outputs |
| **Understand pronouns** | "my budget", "the loan" | Context from earlier turns |

### What the LLM Cannot Do

| Limitation | Example | Why |
|------------|---------|-----|
| Cross-session memory | "Remember last week's calculation" | State resets each session |
| Infinite context | Very long conversations | Token limits apply |
| External memory | "Save this for later" | No persistent storage by default |

---

## Section 8: Visualize State Growth

In [ ]:
print("STATE GROWTH VISUALIZATION")
print("=" * 80)
print("\nCar Purchase Conversation:")
print()

# Count messages by type
human_count = sum(1 for m in car_conversation["messages"] if isinstance(m, HumanMessage))
ai_count = sum(1 for m in car_conversation["messages"] if isinstance(m, AIMessage))
tool_count = sum(1 for m in car_conversation["messages"] if isinstance(m, ToolMessage))

print(f"  👤 HumanMessages:  {human_count}")
print(f"  🤖 AIMessages:     {ai_count}")
print(f"  🔧 ToolMessages:   {tool_count}")
print(f"  ─────────────────────")
print(f"  📊 Total:          {len(car_conversation['messages'])}")

print("\n" + "-" * 80)
print("State growth per turn:")
print("  Turn 1: +1 Human, +2 AI (tool call + response), +1 Tool = 4 messages")
print("  Turn 2: +1 Human, +2 AI, +1 Tool = 4 messages (total: 8)")
print("  Turn 3: +1 Human, +2 AI, +1 Tool = 4 messages (total: 12)")
print("  Turn 4: +1 Human, +1 AI (no tool needed) = 2 messages (total: 14)")

---

## Summary

In this notebook, you learned:

| Concept | Key Takeaway |
|---------|-------------|
| **State Persistence** | Same state object = continued conversation |
| **Message Accumulation** | State grows with each turn, never shrinks |
| **Context Awareness** | LLM sees FULL history every invocation |
| **Reference Resolution** | "that amount", "my budget" resolved from history |
| **Fresh Conversations** | New state object = clean slate |

## Multi-Turn Conversation Pattern

```python
# Initialize
state = {"messages": []}

# Turn N (repeat for each turn)
state["messages"].append(HumanMessage(content="User query"))
state = app.invoke(state)
response = state["messages"][-1].content
```

## Context Capabilities Checklist

```
✅ Reference previous values ("that amount")
✅ Recall past calculations ("what was the EMI?")
✅ Modify parameters ("change to 3 years")
✅ Compare results across turns
✅ Understand pronouns and references
❌ Cross-session memory (resets each session)
❌ Infinite context (token limits apply)
```

---

## 🎉 LangGraph Tutorial Series Complete!

### What You've Learned

| Notebook | Topic | Key Concept |
|----------|-------|-------------|
| 01 | Setup & Validation | Environment, LLM initialization |
| 02 | Getting Started with Tools | @tool decorator, schemas |
| 03 | Currency Converter | Multi-parameter tools |
| 04 | EMI Calculator | Complex calculations, edge cases |
| 05 | Graph Construction | StateGraph, nodes, edges, ToolNode |
| 06 | Single Tool Execution | Basic agent → tool → response |
| 07 | Parallel Execution | Multiple independent tools |
| 08 | Sequential Execution | Dependent tool chains |
| 09 | Conversational Context | Multi-turn conversations |

### Core Principles Mastered

- 🔧 **Tools** extend LLM capabilities with real-world actions
- 📊 **State** enables conversation memory and context
- 🔀 **Routing** creates dynamic, conditional behavior
- 🔄 **Cycles** allow iterative reasoning and tool chains
- 🤖 **LLM Autonomy** - the model orchestrates everything

### Next Steps

- Build custom tools for your specific domain
- Add human-in-the-loop workflows
- Implement sub-graphs for complex tasks
- Integrate LangSmith for observability
- Deploy in production applications

---